In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# @title Install Requirements
from IPython.display import clear_output
!pip install git+https://github.com/hosein-m/adapter-transformers.git@ML_Project2020
clear_output()

In [ ]:
# @title Download GLUE + Additional datasets
""" Script for downloading all GLUE data.
Example usage:
    python download_glue_data.py --data_dir data --tasks all
Note: for legal reasons, we are unable to host MRPC.
You can either use the version hosted by the SentEval team, which is already tokenized,
or you can download the original data from:
https://download.microsoft.com/download/D/4/6/D46FF87A-F6B9-4252-AA8B-3604ED519838/MSRParaphraseCorpus.msi  # noqa
and extract the data from it manually.
For Windows users, you can run the .msi file. For Mac and Linux users, consider an external library
such as 'cabextract' (see below for an example). You should then rename and place specific files in
a folder (see below for an example).
mkdir MRPC
cabextract MSRParaphraseCorpus.msi -d MRPC
cat MRPC/_2DEC3DBE877E4DB192D17C0256E90F1D | tr -d $'\r' > MRPC/msr_paraphrase_train.txt
cat MRPC/_D7B391F9EAFF4B1B8BCE8F21B20B1B61 | tr -d $'\r' > MRPC/msr_paraphrase_test.txt
rm MRPC/_*
rm MSRParaphraseCorpus.msi
"""

import os
import sys
import shutil
import tempfile
import urllib.request
import zipfile
import pandas as pd

TASKS = ["CoLA", "SST", "MRPC", "QQP", "STS", "MNLI", "QNLI", "RTE", "WNLI","SPAM","20NEWSGROUPS","CRAIRLINE","CRMESSAGE", "CRDISASTERS","CRECONOMIC","CREMOTION","CRWARM","CRPOLITICAL",
         "CRBIAS","CRPLTMSG","CRPRMEMOTION","CROPN","NEWSAG","CRUS"]
TASK2PATH = {
    "CoLA": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FCoLA.zip?alt=media&token=46d5e637-3411-4188-bc44-5809b5bfb5f4",  # noqa
    "SST": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8",  # noqa
    "MRPC": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc",  # noqa
    "QQP": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQQP-clean.zip?alt=media&token=11a647cb-ecd3-49c9-9d31-79f8ca8fe277",  # noqa
    "STS": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSTS-B.zip?alt=media&token=bddb94a7-8706-4e0d-a694-1109e12273b5",  # noqa
    "MNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FMNLI.zip?alt=media&token=50329ea1-e339-40e2-809c-10c40afff3ce",  # noqa
    "SNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSNLIv2.zip?alt=media&token=6c15d9df-a597-47d1-9a43-0c605cbc6ef9",  # noqa
    "QNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQNLIv2.zip?alt=media&token=6fdcf570-0fc5-4631-8456-9505272d1601",  # noqa
    "RTE": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FRTE.zip?alt=media&token=5efa7e85-a0bb-4f19-8ea2-9e1840f077fb",  # noqa
    "WNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FWNLI.zip?alt=media&token=068ad0a0-ded7-4bd7-99a5-5e00222e0faf",  # noqa
    "SPAM": "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip",
    "20NEWSGROUPS": "https://drive.google.com/uc?export=download&id=1KJkVWPwW_4GPhk2Oca8dedL9AE5cTVDL",
    "CRAIRLINE" : "https://drive.google.com/uc?export=download&id=1YPwb9c3fWFgXDuq-zUU8hD2AJg090SZa",
    "CRMESSAGE" : "https://drive.google.com/uc?export=download&id=1GiyOi-EdhSeq7uL92j1PXaEipE7quUq_",
    "CRDISASTERS": "https://drive.google.com/uc?export=download&id=1EBo_TgSTFEaPQuPTOD4fVCvl3obCE0Dj",
    "CRECONOMIC" : "https://drive.google.com/uc?export=download&id=1LSm9_WHPb1VicehKHDQbLydgqKhsQ0bR",
    "CREMOTION" : "https://drive.google.com/uc?export=download&id=1UV0f5SIP_g_tw_Qfa2xtq2DSWhV0eDu-",
    "CRWARM" : "https://drive.google.com/uc?export=download&id=1bPVIaVIojnzLZ8cX0Pr8322kG7fXeYgf",
    "CRPOLITICAL": "https://drive.google.com/uc?export=download&id=1s3znAGBwtiM961Xrs2AWlE6EOl8_UcWp",
    "CRBIAS" : "https://drive.google.com/uc?export=download&id=1-pv3hHU6-MBFgv0AE4pH1M1vhCuD_YQq",
    "CRPLTMSG":"https://drive.google.com/uc?export=download&id=1OvZd1YNh0iod07DCW2Z1glEs4d7_2mmZ",
    "CRPRMEMOTION": "https://drive.google.com/uc?export=download&id=1BoPiu4PidZABjDt2BPhCJsQyWE4EKKmP",
    "CROPN": "https://drive.google.com/uc?export=download&id=1rJhCr0Bn56q-qbUEi637rTIsuX4b7qmu",
    "NEWSAG": "https://drive.google.com/uc?export=download&id=17an27uSk7JVdJulq3p3FvR1oz9ty4m2g",
    "CRUS" : "https://drive.google.com/uc?export=download&id=1ym1zO14CBYG7bGqBJStv2vbHkmtbNALp",
    "diagnostic": [
        "https://storage.googleapis.com/mtl-sentence-representations.appspot.com/tsvsWithoutLabels%2FAX.tsv?GoogleAccessId=firebase-adminsdk-0khhl@mtl-sentence-representations.iam.gserviceaccount.com&Expires=2498860800&Signature=DuQ2CSPt2Yfre0C%2BiISrVYrIFaZH1Lc7hBVZDD4ZyR7fZYOMNOUGpi8QxBmTNOrNPjR3z1cggo7WXFfrgECP6FBJSsURv8Ybrue8Ypt%2FTPxbuJ0Xc2FhDi%2BarnecCBFO77RSbfuz%2Bs95hRrYhTnByqu3U%2FYZPaj3tZt5QdfpH2IUROY8LiBXoXS46LE%2FgOQc%2FKN%2BA9SoscRDYsnxHfG0IjXGwHN%2Bf88q6hOmAxeNPx6moDulUF6XMUAaXCSFU%2BnRO2RDL9CapWxj%2BDl7syNyHhB7987hZ80B%2FwFkQ3MEs8auvt5XW1%2Bd4aCU7ytgM69r8JDCwibfhZxpaa4gd50QXQ%3D%3D",  # noqa
        "https://www.dropbox.com/s/ju7d95ifb072q9f/diagnostic-full.tsv?dl=1",
    ],
}

MRPC_TRAIN = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt"
MRPC_TEST = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt"


def download_and_extract(task, data_dir):
    print("Downloading and extracting %s..." % task)
    if task == "SPAM":
      data_file = task
      data_dir = data_dir + "/" + task
    else :
      data_file = "%s.zip" % task
    urllib.request.urlretrieve(TASK2PATH[task], data_file)
    with zipfile.ZipFile(data_file) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(data_file)
    print("\tCompleted!")


def format_mrpc(data_dir, path_to_data):
    print("Processing MRPC...")
    mrpc_dir = os.path.join(data_dir, "MRPC")
    if not os.path.isdir(mrpc_dir):
        os.mkdir(mrpc_dir)
    if path_to_data:
        mrpc_train_file = os.path.join(path_to_data, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(path_to_data, "msr_paraphrase_test.txt")
    else:
        print("Local MRPC data not specified, downloading data from %s" % MRPC_TRAIN)
        mrpc_train_file = os.path.join(mrpc_dir, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(mrpc_dir, "msr_paraphrase_test.txt")
        urllib.request.urlretrieve(MRPC_TRAIN, mrpc_train_file)
        urllib.request.urlretrieve(MRPC_TEST, mrpc_test_file)
    assert os.path.isfile(mrpc_train_file), "Train data not found at %s" % mrpc_train_file
    assert os.path.isfile(mrpc_test_file), "Test data not found at %s" % mrpc_test_file
    urllib.request.urlretrieve(TASK2PATH["MRPC"], os.path.join(mrpc_dir, "dev_ids.tsv"))

    dev_ids = []
    with open(os.path.join(mrpc_dir, "dev_ids.tsv"), encoding="utf8") as ids_fh:
        for row in ids_fh:
            dev_ids.append(row.strip().split("\t"))

    with open(mrpc_train_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "train.tsv"), "w", encoding="utf8"
    ) as train_fh, open(os.path.join(mrpc_dir, "dev.tsv"), "w", encoding="utf8") as dev_fh:
        header = data_fh.readline()
        train_fh.write(header)
        dev_fh.write(header)
        for row in data_fh:
            label, id1, id2, s1, s2 = row.strip().split("\t")
            if [id1, id2] in dev_ids:
                dev_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))
            else:
                train_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))

    with open(mrpc_test_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "test.tsv"), "w", encoding="utf8"
    ) as test_fh:
        header = data_fh.readline()
        test_fh.write("index\t#1 ID\t#2 ID\t#1 String\t#2 String\n")
        for idx, row in enumerate(data_fh):
            label, id1, id2, s1, s2 = row.strip().split("\t")
            test_fh.write("%d\t%s\t%s\t%s\t%s\n" % (idx, id1, id2, s1, s2))
    print("\tCompleted!")


def make_ready_example(task, spam_dir):
  if task == 'SPAM':
      df = pd.read_csv(spam_dir + '/SMSSpamCollection',sep = '\t',header = None, encoding = 'latin-1')
      df.columns = ["label", "text"]
      df.label = pd.factorize(df.label)[0]
      df.iloc[:4459].to_csv(spam_dir + '/train.tsv', sep='\t')
      df.iloc[4459:4459 + 557].to_csv(spam_dir + '/dev.tsv', sep='\t')
      df.iloc[4459 + 557:].to_csv(spam_dir + '/test.tsv', sep='\t')
  
GLUE_DIR = "glue_datasets"
if not os.path.isdir(GLUE_DIR):
    os.mkdir(GLUE_DIR)

spam_dir = os.path.join(GLUE_DIR, "SPAM")
if not os.path.isdir(spam_dir):
    os.mkdir(spam_dir)
for task in TASKS:
    if task == 'MRPC':
        format_mrpc(GLUE_DIR, "")
    else:
        download_and_extract(task, GLUE_DIR)
        if task == 'SPAM':
          make_ready_example(task, spam_dir)
clear_output()
print("Completed")

Completed


In [ ]:
# @title Import Requirements
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
from tqdm import tqdm

from transformers import AutoModelWithHeads, AutoTokenizer, AdapterType
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from transformers.data.data_collator import DefaultDataCollator
from transformers import (
    glue_output_modes,
    glue_tasks_num_labels,
    glue_processors,
    glue_convert_examples_to_features,
)

import torch
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from torch.utils.data.dataloader import DataLoader

if torch.cuda.is_available():    
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# @title Hyperparameters

BATCH_SIZE =  32# @param {type:"integer"}
REDUCTION_FACTOR = 12#@param {type:"integer"}
NON_LINEARITY = "gelu" #@param ["gelu", "relu", "sewish"]
EPOCHS =  10#@param {type:"integer"}
MAX_LENGTH =   128#@param {type:"integer"}

TASK = "rte" #@param ["cola", "sst-2", "mrpc", "sts-b", "qqp", "mnli", "mnli-mm", "qnli", "rte", "wnli","spam","20newsgroups","CRairline","CRmessage","CRdisasters","CReconomic","CRemotion","CRwarm","CRpolitical","CRbias","CRpltmsg","CRprmemotion","CRopn","Newsag","CRus"]
if TASK == "mnli" or TASK == "mnli-mm":
  TASK_DIR = os.path.join(GLUE_DIR, "MNLI" )
elif TASK == "cola": 
  TASK_DIR = os.path.join(GLUE_DIR, "CoLA")
else:
  TASK_DIR = os.path.join(GLUE_DIR, TASK.upper())

num_labels = glue_tasks_num_labels["mnli" if "mnli" in TASK else TASK]
output_mode = glue_output_modes[TASK]
processor = glue_processors[TASK]()
print(num_labels)

LEARNING_RATE =  3e-4 #@param {type:"number"}
WARMUP_RATIO =   0.1 #@param {type:"number"}

SAVED_MODELS_DIR = "saved_models"
if not os.path.isdir(SAVED_MODELS_DIR):
    os.mkdir(SAVED_MODELS_DIR)
if not os.path.isdir(os.path.join(SAVED_MODELS_DIR, TASK)):
    os.mkdir(os.path.join(SAVED_MODELS_DIR, TASK))

2


In [ ]:
# @title Load Tokenizer, Model
pretrained_weights = "bert-base-uncased" 
tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
model = AutoModelWithHeads.from_pretrained(pretrained_weights)
model.add_classification_head(TASK, num_labels=num_labels, layers=2)

model.add_adapter(adapter_name=TASK, adapter_type=AdapterType.text_task, config="houlsby")
model.train_adapter([TASK])
model.set_active_adapters(TASK)


In [ ]:
# @title Example to Features
def example_to_features(mode, tokenizer, max_len, label_list, output_mode):
  if mode == "val":
    examples = processor.get_dev_examples(TASK_DIR)
  elif mode == "test":
    examples = processor.get_test_examples(TASK_DIR)
  elif mode == "train":
    examples = processor.get_train_examples(TASK_DIR)

  features = glue_convert_examples_to_features(
                      examples,
                      tokenizer,
                      max_length=max_len,
                      label_list=label_list,
                      output_mode=output_mode,
                  )
  return features

In [ ]:
# @title Load Datasets
train_dataset = example_to_features("train", tokenizer, MAX_LENGTH, processor.get_labels(), output_mode)
val_dataset = example_to_features("val", tokenizer, MAX_LENGTH, processor.get_labels(), output_mode)
test_dataset = example_to_features("test", tokenizer, MAX_LENGTH, processor.get_labels(), output_mode)

data_collator = DefaultDataCollator()
train_dataloader = DataLoader(train_dataset, 
                              batch_size = BATCH_SIZE,
                              sampler = RandomSampler(train_dataset),
                              collate_fn=data_collator.collate_batch)
val_dataloader = DataLoader(
            val_dataset,
            batch_size = BATCH_SIZE,
            sampler = SequentialSampler(val_dataset),
            collate_fn=data_collator.collate_batch)

test_dataloader = DataLoader(
            test_dataset,
            batch_size = BATCH_SIZE,
            sampler = SequentialSampler(test_dataset),
            collate_fn=data_collator.collate_batch)

total_steps = len(train_dataloader) * EPOCHS

In [ ]:
# @title Compute Metrics
def simple_accuracy(preds, labels):
    return (preds == labels).mean()

def f1(preds, labels):
    return f1_score(y_true=labels, y_pred=preds)

def spearman(preds, labels):
    return spearmanr(preds, labels)[0]

def pearson(preds, labels):
    return pearsonr(preds, labels)[0]

def get_metricName(task_name):
    if task_name == "mrpc" or task_name == "qqp":
        return "f1"
    elif task_name == "cola":
        return "mcc"
    elif task_name == "sts-b":
        return "spearmanr"
    else:
        return "acc"

def glue_compute_metrics(metric_name, preds, labels):
    if output_mode == "classification":
        preds = np.argmax(preds, axis=1)
    elif output_mode == "regression":
        preds = np.squeeze(preds)

    assert len(preds) == len(labels)

    if metric_name == "f1":
        return f1(preds, labels)
    elif metric_name == "mcc":
        return matthews_corrcoef(labels, preds)
    elif metric_name == "spearmanr":
        return spearman(preds, labels)
    else:
        return simple_accuracy(preds, labels)

In [ ]:
# @title Optimizer
optimizer = AdamW(model.parameters(), 
                  lr=LEARNING_RATE, 
                  eps=1e-6)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                num_warmup_steps=int(WARMUP_RATIO * total_steps),
                num_training_steps=total_steps)

In [ ]:
# @title Train & Eval
metric_name = get_metricName(TASK)
history = {'train': {}, 'validation': {}}
history['train']['loss'] = []
history['validation']['loss'] = []
history['validation']['val_' + metric_name] = []
best_avg_val_loss = np.inf
best_avg_val_score = 0

model = model.to(device)
model.zero_grad()

for epoch in range(EPOCHS):

  # Training
  model.train()
  train_loss = 0
  for inputs in train_dataloader:
    for k, v in inputs.items():
      inputs[k] = v.to(device)
    
    loss, logits = model(**inputs, adapter_names=[TASK])
    train_loss += loss.item()
    loss.backward()

    optimizer.step()
    scheduler.step()
    model.zero_grad()

  avg_train_loss = train_loss / len(train_dataloader) 
  print("Epoch: {0}".format(epoch+1))
  print("Average training loss: {0:.2f}".format(avg_train_loss))

  # Evaluating
  model.eval()
  val_loss = 0
  val_score = 0
  for inputs in val_dataloader:
    for k, v in inputs.items():
      inputs[k] = v.to(device)
    
    with torch.no_grad(): 
      loss, logits = model(**inputs, adapter_names=[TASK])
    val_loss += loss.item()
    
    preds = logits.detach().cpu().numpy()
    label_ids = inputs["labels"].detach().cpu().numpy()
    val_score += glue_compute_metrics(metric_name, preds, label_ids)
  
  avg_val_loss = val_loss / len(val_dataloader) 
  avg_val_score = val_score / len(val_dataloader) 
  print("Average validation loss: {0:.2f}".format(avg_val_loss))
  print("Average validation {0}: {1:.3f}".format(metric_name, avg_val_score))

  # Logg and save
  history['train']['loss'].append(avg_train_loss)
  history['validation']['loss'].append(avg_val_loss)
  history['validation']['val_' + metric_name].append(avg_val_score)
  
  if avg_val_score > best_avg_val_score or (avg_val_score == best_avg_val_score and avg_val_loss < best_avg_val_loss):
    best_avg_val_score = avg_val_score
    best_avg_val_loss = avg_val_loss
    
    os.mkdir(os.path.join(SAVED_MODELS_DIR, TASK, str(epoch+1)))
    torch.save(model.state_dict(), os.path.join(SAVED_MODELS_DIR, TASK, str(epoch+1), 'saved_model.pt'))
    print("Model saved as best model")



Epoch: 1
Average training loss: 0.70
Average validation loss: 0.71
Average validation acc: 0.569
Model saved as best model


KeyboardInterrupt: ignored

In [ ]:
# @title Restore the best model
model = AutoModelWithHeads.from_pretrained(pretrained_weights)
model.add_classification_head(TASK, num_labels=num_labels, layers=2)
model.add_adapter(adapter_name=TASK, adapter_type=AdapterType.text_task, config="houlsby")
model.train_adapter([TASK])

BEST_EPOCH = 0#@param {type:"integer"}
model.load_state_dict(torch.load(os.path.join(SAVED_MODELS_DIR, TASK, str(BEST_EPOCH), "saved_model.pt"))
model.set_active_adapters(TASK)

In [ ]:
# @title Test Additional Datasets
test_score = 0
model = model.to(device)
model.eval()
for inputs in tqdm(test_dataloader):
  for k, v in inputs.items():
    inputs[k] = v.to(device)
  
  with torch.no_grad(): 
    _, logits = model(**inputs, adapter_names=[TASK])
    
  test_preds = logits.detach().cpu().numpy()
  label_ids = inputs["labels"].detach().cpu().numpy()
  test_score += glue_compute_metrics(metric_name, test_preds, label_ids)

avg_test_score = test_score / len(test_dataloader) 
print("Average validation {0}: {1:.3f}".format(metric_name, avg_test_score))


# Save Model to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
stop here

In [ ]:
!mkdir /content/drive/My\ Drive/Saved\ Models/$TASK

In [ ]:
!cp -r saved_models/$TASK/?/* /content/drive/My\ Drive/Saved\ Models/$TASK/

#Archived Codes

In [ ]:
# @title Trainable Params
for w in model.named_parameters():
  # if w[1].requires_grad == True:
  print(w[0], end=",\t")
  print(w[1].shape)

bert.embeddings.word_embeddings.weight,	torch.Size([30522, 768])
bert.embeddings.position_embeddings.weight,	torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight,	torch.Size([2, 768])
bert.embeddings.LayerNorm.weight,	torch.Size([768])
bert.embeddings.LayerNorm.bias,	torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight,	torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias,	torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight,	torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias,	torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight,	torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias,	torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight,	torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias,	torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight,	torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias,	torch.Size([768